# Step 1: Data and Model Preparation


In [1]:
exp_name = 'exp_063'

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

In [3]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_2_rpn import GraphBatcher

In [4]:
from src.tensorflow.models.vsp_model_001_abl_01 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_03 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1224)
np.random.seed(1225)

In [6]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'hanwang', 'sg.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'hanwang', 'class_embs.pkl'),
)


In [7]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'hanwang', 'split.pkl'))


In [8]:
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [9]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'coords.pkl'))

In [10]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [11]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [12]:
tf.reset_default_graph()

In [13]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=3,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Use tf.cast instead.


In [14]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training


In [15]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [15]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [16]:
trainer.train (
    max_steps=30000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 1000: loss = 5.159999847412109 (3016.908 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.408, 'PerImagePredMR': 0.941, 'PerImageOneHopMP_at100_iou_0.5': 0.001, 'PerImageOneHopMR_at100_iou_0.5': 0.009, 'PerImageOneHopMP_at50_iou_0.5': 0.002, 'PerImageOneHopMR_at50_iou_0.5': 0.008, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.013, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.072, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.02, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.06}
Validation Results: {'PerImagePredMP': 0.404, 'PerImagePredMR': 0.928, 'PerImageOneHopMP_at100_iou_0.5': 0.001, 'PerImageOneHopMR_at100_iou_0.5': 0.01, 'PerImageOneHopMP_at50_iou_0.5': 0.002, 'PerImageOneHopMR_at50_iou_0.5': 0.009, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.017, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.083, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.025, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.073}
Step 2000: loss = 5.882999897003174 (5971.854 sec)
Training Batch Evaluation Results: {'Per

In [17]:
trainer.train (
    max_steps=20000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 31000: loss = 3.569999933242798 (2960.747 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.526, 'PerImagePredMR': 0.925, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.024, 'PerImageOneHopMP_at50_iou_0.5': 0.006, 'PerImageOneHopMR_at50_iou_0.5': 0.021, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.031, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.189, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.049, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.162}
Validation Results: {'PerImagePredMP': 0.52, 'PerImagePredMR': 0.906, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.028, 'PerImageOneHopMP_at50_iou_0.5': 0.007, 'PerImageOneHopMR_at50_iou_0.5': 0.024, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.032, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.202, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.051, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.173}
Step 32000: loss = 3.7899999618530273 (5904.352 sec)
Training Batch Evaluation Results: 

KeyboardInterrupt: 

In [ ]:
trainer.train (
    max_steps=5000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 41000: loss = 3.2279999256134033 (3015.233 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.517, 'PerImagePredMR': 0.927, 'PerImageOneHopMP_at100_iou_0.5': 0.005, 'PerImageOneHopMR_at100_iou_0.5': 0.033, 'PerImageOneHopMP_at50_iou_0.5': 0.008, 'PerImageOneHopMR_at50_iou_0.5': 0.03, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.034, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.214, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.053, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.179}
Validation Results: {'PerImagePredMP': 0.516, 'PerImagePredMR': 0.913, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.03, 'PerImageOneHopMP_at50_iou_0.5': 0.008, 'PerImageOneHopMR_at50_iou_0.5': 0.028, 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.034, 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.218, 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.053, 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.183}
Step 42000: loss = 3.3469998836517334 (5959.671 sec)
Training Batch Evaluation Results: 

In [ ]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [16]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-45000'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_063/ckpt-45000


In [17]:
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [19]:
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [20]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
Batch 1/808 done in 1 seconds. Spent time for read=0.01 (avg 0.01), processing=1.00 (avg 1.00), write=0.00 (avg 0.00) seconds.
Batch 101/808 done in 5 seconds. Spent time for read=0.01 (avg 0.01), processing=0.04 (avg 0.05), write=0.00 (avg 0.00) seconds.
Batch 201/808 done in 9 seconds. Spent time for read=0.00 (avg 0.01), processing=0.04 (avg 0.04), write=0.00 (avg 0.00) seconds.
Batch 301/808 done in 14 seconds. Spent time for read=0.01 (avg 0.01), processing=0.04 (avg 0.04), write=0.00 (avg 0.00) seconds.
Batch 401/808 done in 18 seconds. Spent time for read=0.01 (avg 0.01), processing=0.04 (avg 0.04), write=0.00 (avg 0.00) seconds.
Batch 501/808 done in 23 seconds. Spent time for read=0.01 (avg 0.01), processing=0.04 (avg 0.04), write=0.00 (avg 0.00) seconds.
Batch 601/808 done in 27 seconds. Spent time for read=0.01 (avg 0.01), processing=0.04 (avg 0.04), write=0.00 (avg 0.00) seconds.
Batch 701/808 done in 32 seconds. Spent time for read=0.01 (avg 0.01), processing=0.0

In [21]:
test_results = test_evaluator.evaluate()
test_results

{'PerImagePredMP': 0.5169912976213498,
 'PerImagePredMR': 0.9184643225170742,
 'PerImageOneHopMP_at100_iou_0.5': 0.0043778766196093605,
 'PerImageOneHopMR_at100_iou_0.5': 0.027631009981767672,
 'PerImageOneHopMP_at50_iou_0.5': 0.0074260297814736035,
 'PerImageOneHopMR_at50_iou_0.5': 0.0244146134574625,
 'PerImageOneHopMP_phrase_at100_iou_0.5': 0.03303500290079289,
 'PerImageOneHopMR_phrase_at100_iou_0.5': 0.19774083200113848,
 'PerImageOneHopMP_phrase_at50_iou_0.5': 0.05198839682846646,
 'PerImageOneHopMR_phrase_at50_iou_0.5': 0.16711022307855114}